In [1]:
import matplotlib.pyplot as plt
from graphics import *
from operator import *
from random import *
import numpy as np
import timeit
from math import *

a = 3
s = 3
r = 3
rr = 20
ro = 70
ra = 300
noise = 0.01
prop = 0.4
weight = 3
bias = np.array([0.0,-1.0])
dev_bias = 3
dTheta = 120
sight_theta = 180


In [41]:
def initialize_agents(speed, N, width, height):
    """
    Initializes our agent set with randomly directed speeds, draws the window and the agents
    """
    seed()

    agents = [np.array([0.0, 0.0]) for i in range(N)]
    #agents = [Point(uniform(0, width), uniform(0, height)) for i in range(N)]
    
    speeds = N*[0,0]
    #speeds = [np.array([0.0, 0.0]) for i in range(N)]
    
    for i in range(N):
        theta = uniform(0, 2 * np.pi)
        #speeds[i][0] = speed * np.cos(theta)
        #speeds[i][1] = speed * np.sin(theta)
        speeds[i]= speed * np.array([np.cos(theta),np.sin(theta)])
        
        theta = uniform(0, 2 * np.pi)
        radius = uniform(0,height/2)
        agents[i] = Point(width/2 + radius*np.cos(theta), height/2 + radius*np.sin(theta))

    return agents, speeds


def initialize_window(agents, width, height):
    win = GraphWin("Swarm", width, height) # size of box
    for agent in agents:
        agent.draw(win)
    win.getMouse()
    return win


def couple_speeds(agents, speeds, a, s, N):
    """
    Simplest model: for each agent, it will give its nearest neighbour a fraction of its speed and re-normalize it
    
    """
    nearest_neighbours = [nearest_neighbour(agent, agents, N) for agent in agents]
    for i in range(N):
        weightedSpeed = a * speeds[nearest_neighbours[i]]
        speeds[i] = speeds[i] + weightedSpeed
        speeds[i] = s * normalized(speeds[i])


def angle(vec1, vec2):  # (Not so) stupid way of doing an angle
    return np.arccos(np.dot(normalized(vec1),normalized(vec2)))
            
def get_distances(agent, agents, N):
    """
    Given one angent and the set of all agents, 
    computes the distances from the first to all of the others
    """
    dists = N * [0.0]
    for i in range(N):
        a, dists[i] = relative_pos(agent, agents[i])
        if dists[i] == 0:
            dists[i] = 0.1
    
    return dists

def relative_pos(agent1, agent2):
    dx = agent2.getX() - agent1.getX()
    dy = agent2.getY() - agent1.getY()

    return np.array([dx, dy]), np.linalg.norm([dx, dy])

def nearest_neighbour(agent, agents, N):
    """
    Returns the index for the agent with smallest Eucledian distance to agent in question
    """
    distances = get_distances(agent, agents, N)
    j = next(i for i in range(N) if agents[i] is agent)
    distances[j] = distances[j-1] + 1

    return distances.index(min(distances))


def softened_angle(speed, newspeed, s, maxTheta):
    theta = angle(speed, newspeed)
    if maxTheta**2 > theta**2:
        return s*newspeed
    else:
        return np.dot(rot_matrix(maxTheta), speed)    

def in_sight_range(rel_pos, speed1, angle_range):
    return 360*angle(speed1, rel_pos)/np.pi < angle_range
    

def noisy_vector(noise):
    return noise * np.array([2 * random() - 1, 2 * random() - 1])
    
def biaser(speeds, N, s, i, prop, bias, dev_bias, weight):
    #bias = np.array([0.0,1.0])
    #Ns has to be integer 
    Ns = int(N * prop)
    gbias = np.random.normal(bias, dev_bias, )
    for i in range(Ns):
        tot_dir = normalized(normalized(speeds[i]) + weight * gbias)
        
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = s*tot_dir
    #bias = np.dot(tot_dir,np.array([[np.cos(rot_bias*i), 0],[0, np.sin(rot_bias*i)]]))
    #return bias

def quality (agents, speeds, N, bias, window, old_cm):
#ACCURACY (DIRECTIONS DISTRIBUTION)
    dev = 0.0
    for i in range(N):
        dev += angle(bias, speeds[i])
    dev_avg = dev/(N*2*(np.pi))
    #print (dev_avg)
    
#ELONGATION: SHAPE OF SWARM
    #can be smarter if we make agents become poses before, more globally in  the code
    poses = [np.array([a.getX(), a.getY()]) for a in agents]
    #Center of Mass
    cm = np.mean(poses,axis=0)
    #Standard Deviation
    std = np.std(poses,axis=0)
    #print (cm, std)

    #Elongation >1 means 
    elong = std[1] / std[0]
    #Drawing it
    #old_cm = Point(cm[0],cm[1])
    #old_cm.draw(window)
    #old_cm.setFill("red")

#GROUP DIRECTION
    #Vector
    group_dir = np.array([cm[0] - old_cm[0], cm[1] - old_cm[1]])                
    #print (group_dir)
    # Norm
    group_dir = np.linalg.norm(group_dir)
    #print (group_dir)
    return cm
    


def rigid_boundary(x_bound, y_bound, agents, speeds, N):
    for i in range(N):
        [dx, dy] = [0, 0]
        [x, y] = [agents[i].getX(), agents[i].getY()]
        if x > x_bound:
            speeds[i][0] = -speeds[i][0]
            dx = x_bound - x         
        
        elif x < 0:
            speeds[i][0] = -speeds[i][0]
            dx = -x
        
        if y > y_bound:
            speeds[i][1] = -speeds[i][1]
            dy = y_bound - y
        
        elif y < 0:
            speeds[i][1] = -speeds[i][1]
            dy = -y

        agents[i].move(dx, dy)
        
        
def periodic_boundary(x_bound, y_bound, agents, speeds, N):  #Changed from rigid boundaries do periodic boundary condition
    [dx, dy] = [0, 0]
    for i in range(N):
        [x, y] = [agents[i].getX(), agents[i].getY()]
        if x > x_bound:
            dx = -x_bound         
        
        elif x < 0:
            dx = x_bound
        
        if y > y_bound:
            dy = -y_bound
        
        elif y < 0:
            dy = y_bound

        agents[i].move(dx, dy)

def next_step(agents, speeds, dt, N):
    dxvec = [dt * speeds[i][0] for i in range(N)]
    dyvec = [dt * speeds[i][1] for i in range(N)]
    for i in range(N):
        agents[i].move(dxvec[i], dyvec[i])
        

def normalized(vector):
    if vector[0] == vector[1] == 0:
        return vector
    return vector / np.linalg.norm(vector)


def rot_matrix(theta, unit = "None"):
    if unit in ["None", "deg"]:
        c, s = np.cos(pi * theta / 180), np.sin(pi * theta / 180)
    elif unit == "rad":
        c, s = np.cos(theta), np.sin(theta)
    return np.array([[c, -s],[s, c]])

def warn_me_args(N_steps, a, dt, N, width, height, s, rr, ro, ra, noise, prop, weight, bias, dTheta):
    if dt*s < rr:
        print("Warning - step length bigger the repulsion radius.")
        

        
def virtualizer (current, agents, h, w, N):
    lower_limit = current.getY() - h / 2
    upper_limit = current.getY() + h / 2
    left_limit = current.getX() - w / 2
    right_limit = current.getX() + w / 2
    
    vagents=[np.array([agent.getX(),agent.getY()]) for agent in agents]
    virtuals = N * [0.0, 0.0]
    vvirtuals = N * [Point(0.0, 0.0)]

    for j in range(N):
        #make more compact!!!
        virtuals[j]=vagents[j]
        
        #newcopy = agents[i].clone()
        candidates = [vagents[j],
                      vagents[j]+[w,0],vagents[j]+[-w,0],vagents[j]+[w,h],vagents[j]+[w,-h],
                      vagents[j]+[-w,h],vagents[j]+[-w,-h],vagents[j]+[0,h],vagents[j]+[0,-h]]
        #print candidates
        
        #virtuals[j] = next((cand for cand in candidates if lower_limit < cand[1] < upper_limit and left_limit < cand[0] < right_limit),False
        for i in range(9):
            if lower_limit < candidates[i][1] < upper_limit and left_limit < candidates[i][0] < right_limit:
                virtuals[j]=candidates[i]
            
        vvirtuals[j] = Point(virtuals[j][0],virtuals[j][1])
        
    return vvirtuals



In [ ]:
#COUZIN MODEL IMPLEMENTED WITH REPULSE, ATRACT AND ORIENT ZONES SEPARATED (1ST PAPER)
def couzin(agents, speeds, N, width, height, s, noise, dTheta, rr, ro, ra, sight_range, model2, roa, atract, orient):
    
    if not model2:
        atract = orient = 1
    # watch only particles in repulsion zone
    for i in range(N):
        r_dir = np.array([0.0, 0.0])
        o_dir = np.array([0.0, 0.0])
        a_dir = np.array([0.0, 0.0])
        repulsion_flag = False    
        
        virtuals = virtualizer(agents[i], agents, height, width, N)
        
        for j in range(N):     
            if i == j:
                #Eliminate the i-i interaction
                continue

            rel_pos, distance = relative_pos(agents[i], virtuals[j])
            
            if in_sight_range(rel_pos, speeds[i], sight_range):
                if distance < rr:
                    rel_pos = normalized(rel_pos)
                    r_dir = r_dir + rel_pos
                    repulsion_flag = True

                elif not repulsion_flag:
                    
                    if model2: # Couzin 2
                        if distance < roa:
                            o_dir = o_dir + speeds[j]
                            rel_pos = normalized(rel_pos)
                            a_dir = a_dir + rel_pos

                    else: # Couzin 1
                        if distance < ro:
                            o_dir = o_dir + speeds[j]

                        elif distance < ra:
                            rel_pos = normalized(rel_pos)
                            a_dir = a_dir + rel_pos

        #Out of for (j), we treat now the resulting direction vector
        if repulsion_flag:
            tot_dir =  normalized(- r_dir)
        else:
            tot_dir = orient * normalized(o_dir) + atract * normalized(a_dir)
            tot_dir = normalized(tot_dir)
            
        tot_dir = normalized(normalized(tot_dir) + noisy_vector(noise))
        
        #avoid pts stoping when not interacting
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = softened_angle(speeds[i], tot_dir, s, dTheta)
    return

    
def vicsek(agents, speeds, N, s, noise, r): # s=speed, noise= letter csi temperature factor, r=radius of interaction
    # consider only particles within 'r' from pt_i, align pt_i with v_avg
    for i in range(N):
        distances = get_distances(agents[i], agents)
        tot_dir = np.array([0.0, 0.0])
        
        for j in range(N):
            if distances[j] < r:
                tot_dir = tot_dir + speeds[j]

        tot_dir = s * normalized(normalized(tot_dir) + noise * np.array([(-1)+2*random(),(-1)+2*random()]))
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = tot_dir
    return

##MILL MODEL
def mill (dt, agents, speeds, N, width, height, cr, ca, lr, la, alpha, beta, mass):
    clr = cr / lr
    cla = ca / la
    for i in range(N):
        u_dir = np.array([0.0, 0.0])
        propulsion = np.array([0.0, 0.0])
        friction = np.array([0.0, 0.0])
        grad_U = np.array([0.0, 0.0])
        
        #virtuals = virtualizer(agents[i], agents, height, width, N)
        
        for j in range(N):   #Duality interactions, by the Morse potential  
            if i == j:
                #Eliminate the i-i interaction
                continue

            rel_pos, distance = relative_pos(agents[i], agents[j])
            u_dir = normalized(rel_pos)
            grad_U = grad_U + u_dir * (clr*np.exp(-distance / lr) - cla *np.exp(- distance / la))
            
        
        propulsion = alpha * speeds[i] # self-propulsion propto alpha
        norm = (np.linalg.norm(speeds[i]))
        friction =  beta *((norm)**2) * speeds[i] #friction force propto beta
        
        #print i,  agents[i]
        #print speeds[i], norm
        #print propulsion, friction, grad_U
        
        
        d_speed = (propulsion - friction - grad_U)/mass
        speeds[i]= speeds[i]+dt*d_speed
        #print u_dir
    
   # print speeds[N-1], propulsion, friction, grad_U
    return

def run(N_steps, N, s, dt, width, height):#N_steps, a, dt, N, width, height, s, rr, ro, ra, noise, prop, weight, bias, dev_bias, dTheta, sight_theta):
    """
    Simulates motion of swarm. Recieves following parameters:

    N_steps  - number of steps to perform
    a -  coupling between neighbouring points
    dt - time step to be used 
    N - number of points to be used
    width & heigth of window
    s - module of speed throughout agents

    """
    
    agents, speeds = initialize_agents(s, N, width, height)
    window = initialize_window(agents, width, height)
    old_cm=np.array([0.0,0.0])
    bias = np.array([0.0,-1.0])
    for i in range(N_steps):
        
        #print 'step', i
        
        next_step(agents, speeds, dt, N)
        start = timeit.default_timer()
        ## MODEL
        #couple_speeds(agents, speeds, a, s, N)
        #vicsek(agents, speeds, N, s, 0.1, r = 500)
        #couzin(agents, speeds, N, width, height, s, noise, dTheta, rr, ro, ra, sight_theta, 1, 200, 1.5, 1) #(...)last 4 parameters: model2, roa, orient, atract
        #mill (dt, agents, speeds, N, width, height, cr=1.0, ca=0.5, lr=0.5, la=3.0, alpha=1, beta=0.5)
        #mill (dt, agents, speeds, N, width, height, cr=40, ca=5, lr=5 ,la=100.0, alpha=0.2, beta=0.5) #equilibrium for 5 particles
        #mill (dt, agents, speeds, N, width, height, cr=100, ca=80, lr=50 ,la=200.0, alpha=1, beta=0.01) #stable swarm, but no clear mill
        #mill (dt, agents, speeds, N, width, height, cr=100, ca=110, lr=90, la=200.0, alpha=1.6, beta=0.5) #
        
        #mill (dt, agents, speeds, N, width, height, cr=100, ca=150, lr=50, la=200.0, alpha=0.8, beta=0.01) # AAAALMOST a nice mill
        #mill (dt, agents, speeds, N, width, height, cr=100, ca=150, lr=80, la=200.0, alpha=0.8, beta=0.01) # more than AAAALMOST, with less pts, it reaches faster  
        #mill (dt, agents, speeds, N, width, height, cr=100, ca=150, lr=80, la=200.0, alpha=1.0, beta=0.01) # we're there! N=30, s=5, dt=0.1, Radius=height/4  
        #mill (dt, agents, speeds, N, width, height, cr=100, ca=150, lr=80, la=200.0, alpha=1.2, beta=0.01, mass=3) # we're there! N=30, s=5, dt=0.1, Radius=height/4  
        
        #mill (dt, agents, speeds, N, width, height, cr=100, ca=150, lr=80, la=200.0, alpha=1.0, beta=0.01, mass=1) 
        # Region I, nice milling clumps
        
        #mill (dt, agents, speeds, N, width, height, cr=120, ca=250, lr=120, la=250.0, alpha=1.0, beta=0.01, mass=1) 
        # Region II, really nice ring
        
        #mill (dt, agents, speeds, N, width, height, cr=120, ca=250, lr=230, la=250.0, alpha=1.0, beta=0.01, mass=1) 
        # Region III, clumps into very small ring (could be better)
        # mill (dt, agents, speeds, N, width, height, cr=120, ca=250, lr=230, la=250.0, alpha=3.0, beta=0.01, mass=4) 
        # Region III, nice milling clumps (very strong clumping) (improved with ++alpha, ++mass)
        
        #mill (dt, agents, speeds, N, width, height, cr=120, ca=250, lr=230, la=150.0, alpha=4.0, beta=0.01, mass=4) 
        # Region IV, nice milling in almost perfect ring (improved with ++alpha, ++mass)
        # broke down after long time because pts fall into few individual clumps
        
        #mill (dt, agents, speeds, N, width, height, cr=230, ca=150, lr=230, la=150.0, alpha=1.0, beta=0.01, mass=1) 
        # Region V, pts spread out nicely (only repulsion), no apparent stable structure, 1/4 circle at corners
        
        #mill (dt, agents, speeds, N, width, height, cr=250, ca=100, lr=150, la=200.0, alpha=1.0, beta=0.01, mass=1) 
        # Region VI, (l>1/sqrt(C))  pts spread out (repulsion + atraction), no apparent stable structure because of no special alpha/beta
        
        #mill (dt, agents, speeds, N, width, height, cr=250, ca=100, lr=150, la=200.0, alpha=0.1, beta=0.1, mass=1) 
        # Region VI,(l>1/sqrt(C))  pts almost stop in a struct because beta increased
        # ERROR for too high beta 
        
        #mill (dt, agents, speeds, N, width, height, cr=25, ca=10, lr=15, la=20, alpha=0.10, beta=0.1, mass=1) 
        # Region VI,(l>1/sqrt(C)) kindof swarm  (yet to find: pts in small crystal struc!)
        
        #mill (dt, agents, speeds, N, width, height, cr=25, ca=10, lr=10, la=20.0, alpha=1.0, beta=0.01, mass=1) 
        # Region VII,(l<1/sqrt(C)) alpha >> beta, gaseous phase
        
        #mill (dt, agents, speeds, N, width, height, cr=25, ca=10, lr=10, la=20.0, alpha=0.1, beta=0.5, mass=1) 
        # Region VII,(l<1/sqrt(C)) alpha << beta, crystalline struct
        
        #mill (dt, agents, speeds, N, width, height, cr=110, ca=100, lr=20, la=100.0, alpha=4.0, beta=0.06, mass=3) 
        # Region VII,(l<1/sqrt(C)) alpha ~ beta, best vortex in this region, decreasing in size with increasing N
        
        #With exp part, doesnt make sense!!!
        #mill (dt, agents, speeds, N, width, height, cr=1*np.exp(-1/100), ca=0.5*np.exp(-1/100), lr=50, la=200, alpha=0.16, beta=0.05, mass=1) 
        # Region VII,(l<1/sqrt(C)) alpha ~ beta, NO nice vortex,
        
        #mill (dt, agents, speeds, N, width, height, cr=100, ca=50, lr=50, la=200, alpha=1.6, beta=0.05, mass=1) 
        # Region VII,(l<1/sqrt(C)) alpha ~ beta, NO vortex, parameters from paper.
        
        
        
        #interaction(agents, speeds)
        
        #biaser(speeds, N, s, i, prop, bias, dev_bias, weight)
        
        #INFORMATION TRANSFER: SHAPE & DIRECTION & ALIGNMENT QUALITY 
        #point_cm.undraw()
        #old_cm = quality(agents, speeds, N, bias, window, old_cm)
        
        ## BOUNDARY CONDITIONS
        rigid_boundary(width, height, agents, speeds, N)
        #periodic_boundary(width, height, agents, speeds, N)

        #time.sleep(0.02)
        stop = timeit.default_timer()
        #print stop - start
    window.close()
    return

In [ ]:
run(100000, 50, 5, 0.3, 900, 700)

[ 0.43265359  0.90156024]
[-0.84497229 -0.53481008]
[ 0.58314194 -0.81237029]
[ 0.95623223 -0.29260883]
[ 0.68061685 -0.73263955]
[ 0.42214041  0.90653046]
[-0.1545704   0.98798178]
[ 0.91697304  0.39894919]
[-0.93058776 -0.36606888]
[ 0.69881721 -0.71530029]
[ 0.87048995 -0.4921862 ]
[ 0.99985848 -0.016823  ]
[ 0.94007212  0.34097567]
[ 0.29372424 -0.9558902 ]
[ 0.95391266 -0.30008437]
[ 0.84888531  0.52857708]
[-0.56759797 -0.82330586]
[ 0.98650021  0.16376001]
[ 0.99685663 -0.0792266 ]
[ 0.95419733  0.29917795]
[ 0.7996989  -0.60040125]
[ 0.423422   -0.90593256]
[ 0.99991239 -0.01323652]
[ 0.98759079  0.15704915]
[-0.7893906   0.61389126]
[ 0.92623061 -0.37695737]
[ 0.46565196 -0.88496794]
[ 0.70277318 -0.71141399]
[ 0.30456737 -0.9524908 ]
[ 0.86973694 -0.49351561]
[ 0.19339604 -0.98112077]
[ 0.51369436 -0.85797325]
[ 0.9786795   0.20539336]
[ 0.5968017   0.80238877]
[ 0.49041112 -0.87149121]
[ 0.92907001  0.36990393]
[-0.0562928 -0.9984143]
[ 0.5131002  -0.85832872]
[ 0.99653465 -

[ 0.91915157  0.39390403]
[-0.83313092  0.55307583]
[ 0.94638648 -0.32303658]
[ 0.43931583 -0.89833268]
[ 0.71330398 -0.70085479]
[ 0.11817831 -0.99299239]
[ 0.9170325  -0.39881248]
[ 0.03963043 -0.99921441]
[ 0.43240167 -0.90168109]
[ 0.95614     0.29291005]
[ 0.43088051  0.90240899]
[ 0.34617376 -0.93817042]
[-0.241632    0.97036796]
[-0.34102853 -0.94005295]
[ 0.35947559 -0.93315449]
[ 0.99855356  0.05376612]
[ 0.91750582 -0.39772236]
[ 0.98951982  0.14439711]
[ 0.98877553 -0.14940867]
[ 0.51554761  0.85686093]
[ 0.84051004 -0.54179597]
[ 0.96297386 -0.26959476]
[ 0.9755115  -0.21994842]
[-0.23054846  0.97306085]
[ 0.98710092  0.16009929]
[ 0.68556652  0.72800999]
[-0.68556652 -0.72800999]
[ 0.25743937  0.96629445]
[-0.98098452 -0.19408599]
[ 0.46032783 -0.887749  ]
[ 0.99995381  0.00961154]
[ 0.59537002 -0.80345164]
[-0.01801193  0.99983777]
[-0.35374619 -0.93534145]
[ 0.56024816  0.82832482]
[ 0.96057462 -0.27802231]
[ 0.60100932 -0.79924201]
[ 0.91912887 -0.393957  ]
[ 0.99244659

[ 0.7846759   0.61990622]
[ 0.99806085  0.06224587]
[ 0.13691148  0.99058329]
[-0.60492927  0.79627921]
[-0.48670481  0.8735665 ]
[-0.34319686  0.9392635 ]
[-0.51340728 -0.85814507]
[-0.78845942  0.61508678]
[ 0.19591309  0.98062126]
[-0.19591309 -0.98062126]
[ 0.021821    0.99976189]
[-0.99152945  0.12988206]
[-0.91576316 -0.4017186 ]
[-0.69112718  0.72273316]
[ 0.24536193 -0.96943155]
[-0.28664601  0.95803657]
[-0.68615854 -0.72745204]
[-0.57629779  0.81723978]
[ 0.99555239  0.0942095 ]
[-0.89266535 -0.45072006]
[-0.58538083  0.81075846]
[-0.91863393  0.39510973]
[-0.86291308  0.50535237]
[-0.25791683  0.96616712]
[-0.58328638  0.81226658]
[-0.45079863  0.89262567]
[-0.70237252  0.71180956]
[ 0.705453   0.7087567]
[ 0.77875941  0.62732271]
[ 0.44499705  0.89553203]
[-0.6130738   0.79002564]
[-0.61899285 -0.78539662]
[ 0.74654077  0.66533968]
[-0.70909452  0.70511344]
[-0.8226465  0.5685532]
[-0.48022218  0.87714688]
[-0.22855481  0.97353105]
[-0.67668285  0.73627462]
[-0.93772245 -0.

[ 0.29412688  0.95576638]
[-0.59798068 -0.80151051]
[ 0.63043076 -0.77624549]
[ 0.90887095  0.41707745]
[-0.79613339  0.60512117]
[-0.46121105  0.88729047]
[ 0.03565245  0.99936425]
[-0.99961381 -0.02778909]
[-0.79264993  0.60967704]
[-0.63329526  0.77391027]
[-0.99845606 -0.05554728]
[ 0.05734008 -0.9983547 ]
[-0.45348631 -0.89126324]
[ 0.87294235  0.48782339]
[-0.87394902 -0.4860176 ]
[ 0.74684717 -0.66499572]
[-0.19131938  0.98152784]
[ 0.87929893  0.4762703 ]
[-0.91616898 -0.40079222]
[-0.84323589  0.53754371]
[-0.48714352  0.87332193]
[-0.4333252   0.90123763]
[ 0.87749006  0.47959482]
[ 0.60189825  0.79857279]
[ 0.99136059  0.13116468]
[ 0.71892912  0.6950834 ]
[-0.60484431  0.79634374]
[ 0.5205151   0.85385246]
[ 0.20118035  0.97955422]
[ 0.72321622  0.69062168]
[-0.39125074  0.92028412]
[ 0.31071001  0.95050475]
[ 0.20933702  0.97784355]
[-0.88466215 -0.46623265]
[-0.94464468  0.32809514]
[-0.71235114 -0.70182323]
[ 0.97064142 -0.24053113]
[ 0.8849197   0.46574362]
[ 0.93131909

[ 0.71259726 -0.70157334]
[ 0.55874637 -0.82933859]
[ 0.79471879 -0.60697779]
[ 0.24686439 -0.96905004]
[ 0.81080208 -0.58532041]
[-0.025825   -0.99966648]
[ 0.025825    0.99966648]
[ 0.9997763  -0.02115053]
[-0.91943409 -0.39324415]
[ 0.4342907  -0.90077277]
[ 0.69966533 -0.71447073]
[-0.38358501 -0.92350557]
[ 0.82103465 -0.57087837]
[-0.33122942 -0.94355025]
[ 0.29614553 -0.95514283]
[ 0.54238958  0.8401271 ]
[ 0.69330799 -0.7206414 ]
[ 0.71601907 -0.69808072]
[ 0.31971525 -0.94751367]
[ 0.98644817  0.1640732 ]
[ 0.69635827 -0.71769434]
[ 0.86252991 -0.50600608]
[ 0.84353388 -0.53707596]
[ 0.69099007 -0.72286425]
[ 0.13597245 -0.99071262]
[-0.25160628 -0.96782967]
[-0.22635841 -0.97404408]
[ 0.83311329 -0.55310238]
[ 0.90764767 -0.4197329 ]
[-0.25838753 -0.96604135]
[ 0.997963    0.06379539]
[-0.88767893  0.46046295]
[ 0.72095821 -0.69297854]
[ 0.71218927 -0.70198749]
[ 0.5574694  -0.83019749]
[ 0.02075144 -0.99978467]
[ 0.71128266 -0.7029061 ]
[ 0.12301937 -0.99240427]
[ 0.7796277 

...

[-0.84361782 -0.5369441 ]
[-0.88910153 -0.45771003]
[ 0.88910153  0.45771003]
[ 0.52429709 -0.85153541]
[-0.99833787 -0.05763239]
[ 0.62280056  0.78238064]
[-0.90895974 -0.4168839 ]
[-0.74418148 -0.66797749]
[ 0.88288736  0.46958483]
[-0.9997118   0.02400645]
[ 0.34566598 -0.93835762]
[-0.3074531   0.95156323]
[-0.72079197 -0.69315145]
[-0.7999766  0.6000312]
[ 0.66635239  0.74563697]
[-0.60264399 -0.79801017]
[-0.76509292 -0.64391989]
[-0.98782406 -0.15557518]
[ 0.32294546 -0.94641758]
[-0.93894938 -0.34405533]
[-0.95186502 -0.30651751]
[-0.99113701 -0.13284364]
[ 0.60886826  0.79327136]
[ 0.60915386 -0.79305206]
[-0.79144899  0.61123522]
[-0.72512466 -0.68861762]
[-0.98067456  0.19564613]
[-0.76991906  0.63814155]
[-0.99898792  0.04497932]
[-0.83937283 -0.54355611]
[-0.92403752 -0.38230179]
[-0.87038004 -0.49238053]
[-0.30236271 -0.95319295]
[ 0.16646497 -0.98604737]
[-0.92557781 -0.37855742]
[-0.8845571   0.46643191]
[-0.31435067 -0.94930693]
[-0.71930746 -0.69469186]
[ 0.37585028 -

[ 0.86617333  0.49974369]
[ 0.15045999  0.9886161 ]
[ 0.70066148  0.71349386]
[-0.41108035  0.91159912]
[ 0.35031471  0.93663205]
[ 0.66075637  0.75060044]
[-0.29276714  0.95618377]
[ 0.48099804  0.87672167]
[ 0.73945205  0.67320923]
[ 0.10918613  0.99402132]
[ 0.9998815  -0.01539413]
[ 0.59884246  0.80086685]
[ 0.17696247  0.9842176 ]
[ 0.40677965  0.91352631]
[ 0.4675275   0.88397853]
[ 0.78064765  0.6249714 ]
[-0.04142518  0.99914161]
[ 0.71422102 -0.69992024]
[ 0.40024902  0.91640642]
[ 0.3909144   0.92042704]
[-0.3909144  -0.92042704]
[-0.62195863  0.7830501 ]
[ 0.51167492 -0.85917913]
[-0.86415577 -0.50322442]
[ 0.79307487  0.60912416]
[ 0.41154505  0.91138942]
[ 0.26064196  0.96543553]
[ 0.62420924  0.78125721]
[-0.60184294  0.79861447]
[-0.97753676 -0.21076499]
[ 0.9369297   0.34951786]
[-0.64824358  0.76143303]
[ 0.20893016  0.97793056]
[ 0.23839031  0.97116943]
[-0.22352675  0.9746978 ]
[ 0.43330854  0.90124564]
[-0.98311339 -0.18299741]
[ 0.63491009  0.77258604]
[ 0.27071927

[ 0.69532442  0.71869601]
[-0.2736119  -0.96184018]
[-0.89637745 -0.44329162]
[ 0.99869892  0.05099486]
[-0.82344123 -0.56740157]
[-0.98485775 -0.17336439]
[-0.1550953   0.98789951]
[-0.66284791 -0.74875406]
[ 0.16097502 -0.98695848]
[-0.2262602 -0.9740669]
[-0.73463794 -0.67845936]
[ 0.47646439 -0.87919377]
[-0.69411345 -0.71986563]
[-0.78117192 -0.62431596]
[-0.68663864 -0.72699888]
[-0.78548033 -0.61888663]
[-0.36167226 -0.9323053 ]
[-0.56640585 -0.82412645]
[-0.99566415 -0.09302093]
[-0.77765945 -0.62868576]
[ 0.57860455 -0.81560822]
[ 0.25113725 -0.96795149]
[-0.87478539  0.4845106 ]
[-0.77317583 -0.63419172]
[-0.99773021 -0.06733819]
[-0.85064617  0.52573862]
[-0.83852623 -0.54486124]
[-0.80789838 -0.58932182]
[-0.48345395 -0.87536979]
[-0.82678096 -0.562524  ]
[-0.79079013 -0.61208739]
[-0.84274181 -0.53831798]
[-0.99964498 -0.0266442 ]
[-0.9754706  -0.22012976]
[-0.42288238 -0.90618458]
[ 0.80056133  0.59925083]
[ 0.18765166 -0.98223564]
[-0.69107523 -0.72278284]
[-0.84288764 -

[ 0.69143804  0.72243576]
[ 0.98599582 -0.16677001]
[ 0.85238672  0.52291192]
[ 0.99993334  0.01154595]
[ 0.90184908  0.4320512 ]
[-0.27937615  0.96018174]
[ 0.26685135  0.9637377 ]
[ 0.58551383  0.81066242]
[ 0.95750626  0.28841247]
[-0.14214757  0.98984548]
[ 0.81804266 -0.57515755]
[ 0.87892285  0.47696397]
[ 0.74692022  0.66491367]
[-0.74692022 -0.66491367]
[ 0.05176492  0.9986593 ]
[ 0.97030974  0.24186567]
[-0.26758826 -0.96353335]
[ 0.82224927  0.56912752]
[ 0.76183366 -0.64777271]
[-0.17188876  0.98511637]
[ 0.71701292  0.69705988]
[-0.52283239  0.85243551]
[ 0.95036567 -0.31113516]
[ 0.81786593  0.57540882]
[ 0.80618346 -0.59166564]
[ 0.65148136  0.75866464]
[ 0.99641771 -0.08456795]
[-0.1779008   0.98404843]
[ 0.88986884  0.45621646]
[-0.87392261 -0.4860651 ]
[ 0.97151492  0.2369784 ]
[ 0.94216932 -0.33513725]
[ 0.15041099  0.98862355]
[-0.6037265  -0.79719151]
[-0.47611214  0.87938458]
[-0.83686064  0.54741599]
[ 0.63219442  0.7748098 ]
[ 0.9991003  -0.04240972]
[-0.81117348

[-0.82642842  0.5630418 ]
[ 0.85642797  0.51626653]
[-0.64319818 -0.76569975]
[-0.00564196 -0.99998408]
[-0.87646406 -0.48146729]
[ 0.5745975   0.81843614]
[ 0.12106711 -0.99264432]
[-0.97023975  0.24214629]
[-0.84485267 -0.53499904]
[ 0.88914301  0.45762944]
[ 0.35445148 -0.93507441]
[ 0.28319383 -0.9590627 ]
[ 0.02713664 -0.99963173]
[-0.94251442 -0.33416549]
[-0.86596132  0.50011097]
[ 0.99975743  0.02202441]
[-0.99804854  0.06244283]
[-0.96016747 -0.27942517]
[ 0.01468255  0.99989221]
[-0.69227734 -0.72163154]
[-0.99999666  0.00258473]
[-0.31456217 -0.94923687]
[-0.80796657 -0.58922833]
[ 0.5022181  -0.86474099]
[-0.681005   -0.73227876]
[ 0.52710634 -0.84979933]
[-0.95947067  0.28180851]
[-0.92019146  0.39146862]
[-0.35854566 -0.93351219]
[-0.89698718 -0.44205656]
[-0.99903166  0.04399717]
[ 0.52706826 -0.84982295]
[ 0.20893834 -0.97792882]
[-0.13743584 -0.99051067]
[-0.99885428  0.04785532]
[ 0.42703072 -0.90423711]
[ 0.76232688 -0.64719219]
[-0.55114959 -0.83440645]
[ 0.29685466

KeyboardInterrupt: 